### Exercício 1 - Pedro Gonçalves (A82313) & Roberto Cachada (A81012)

Para este exercício era pedido a criação de dois agentes, ***Emitter*** e ***Receiver***, de modo a estabelecer uma comunicação privada entre ambos, utilizando para isso a *package* ***Cryptography***. Para tal, foram criadas diferentes funções de modo a cumprir esse objetivo. Essas funções serão descritas em baixo.

##### Alínea a)

* Para resolver esta primeira alínea, foi criada a função ***genNounce***. Esta função, após receber o tamanho do *nounce* a gerar, em *bytes*, obtém uma sequência de caracteres aleatoriamente gerados com esse tamanho, retornando-a em seguida ao utilizador.

In [1]:
import os
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

def genNounce(tamanho): #tamanho em bytes
    nounce = os.urandom(tamanho)
    return nounce

##### Alínea b)

- Nesta alínea foram criadas as funções ***hmac***, ***encrypt*** e ***decrypt***. 
- A ***hmac*** serve para garantir a autenticação de texto limpo e também verificar a autenticidade do mesmo, utilizando para tal uma função de *hash* criptográfica juntamente com uma chave para cumprir esse objetivo.
- Já a funcão ***encrypt*** serve para encriptar/autenticar o texto limpo utilizando a cifra *AES (Advanced Encryption Standard)*, em modo *GCM (Galois Counter Mode)*. Já a função ***decrypt*** serve para decifrar os resultados produzidos pela função ***encrypt*** e verificando em simultâneo a sua autenticidade.
- Foi escolhido o modo *GCM (Galois Counter Mode)* pois o grupo considerou ser dos melhores modos contra ataques de IV, desde que seja garantido que o mesmo se trate de um *nounce*, o que acontece com a nossa implementação. As razões para as quais é necessário que o IV se trate de um *nounce* são apresentadas de seguida:
  - Como a encriptação em modo *GCM* consiste em **TextoCifrado = TextoLimpo $\oplus$ F(Chave, IV)**. Caso se encripte dois textos limpos diferentes usando o mesmo par Chave/IV, **TC1 = TL1 $\oplus$ F(Chave, Iv)** e **TC2 = TL2 $\oplus$ F(Chave, Iv)** é possível através de TC1 e TC2 obter TL1 e TL2 fazendo o XOR de  **TC1 $\oplus$ TC2 = TL1 $\oplus$ TL2**, obtendo assim os textos limpos *XORed*.

In [2]:
from cryptography.hazmat.primitives import hashes, hmac

def mac(chave, textolimpo, tag=None):
    h = hmac.HMAC(chave,hashes.SHA256(),default_backend())
    h.update(textolimpo)
    if tag == None: #Se não existir tag, autentica o texto limpo
        return h.finalize()
    h.verify(tag) #Caso contrário verifica a autenticidade da tag

In [3]:
def encrypt(chave, textolimpo):
    iv = genNounce(12) #gera um IV de 12 bytes

    #Criação de um objeto AES-GCM através da chave e do IV 
    encryptor = Cipher(
        algorithms.AES(chave), #Cifra AES
        modes.GCM(iv), #Modo GCM
        backend=default_backend()
    ).encryptor()

    #Encripta o texto limpo
    ciphertext = encryptor.update(textolimpo) + encryptor.finalize()

    return (iv, ciphertext, encryptor.tag)

def decrypt(chave, iv, ciphertext, tag):
    #Criação de um objeto AES-GCM através da chave, do IV
    decryptor = Cipher(
        algorithms.AES(chave),
        modes.GCM(iv,tag),
        backend=default_backend()
    ).decryptor()

    #Retorna o texto limpo
    return decryptor.update(ciphertext) + decryptor.finalize()

##### Alínea c)

- Nesta alínea foi criada a função ***sts***, onde está definido o protocolo de troca de chaves ***Diffie-Hellman*** e a autenticação dos agentes atráves do esquema de assinaturas ***DSA***.
- Para tal, são criados parámetros ***Diffie-Hellman*** e ***DSA***, dos quais são geradas chaves públicas e privadas para ambos. Em seguida é executado o processo de troca de chaves ***Diffie-Hellman***, sendo em simultâneo verificada a autenticidade dos agentes envolvidos através do esquema de assinaturas ***DSA***.

In [5]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

#São gerados os parámetros
dhParameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())
dsaParameters = dsa.generate_parameters(key_size=2048,backend=default_backend())

In [6]:
salt = os.urandom(16) #Salt partilhado

def sts(agente, conn):
     #--DH--
    chavePrivadaDH = dhParameters.generate_private_key() #Gera uma chave privada DH
    chavePublicaDH = chavePrivadaDH.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo) #Gera uma chave pública DH e serializa a mesma

    #--DSA--
    chavePrivadaDSA = dsaParameters.generate_private_key() #Gera uma chave privada DSA
    chavePublicaDSA = chavePrivadaDSA.public_key().public_bytes( 
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo) #Gera uma chave Pública DSA e serializa a mesma
    
    assinatura = chavePrivadaDSA.sign(chavePublicaDH, hashes.SHA256()) #Agente assina a sua chave DH pública
    
    #É criada uma stream de dados(dicionário), que contém ambas as chaves públicas do agente e a sua assinatura DSA
    dados = {'CP_DH' : chavePublicaDH, 'CP_DSA' : chavePublicaDSA, 'Assinatura' : assinatura} 
    conn.send(dados) #É enviada a stream de dados
    
    dados = conn.recv() #É recebida a stream de dados do outro agente
    peerDH_PK_Bytes = dados['CP_DH']
    peerDSA_PK_Bytes = dados['CP_DSA']
    peerSignature = dados['Assinatura']
    
    # Chaves recebidas são deserializadas
    peerDH_PK = serialization.load_pem_public_key(peerDH_PK_Bytes, backend=default_backend())
    peerDSA_PK = serialization.load_pem_public_key(peerDSA_PK_Bytes, backend=default_backend())
    
    try: #É verificada a validade da assinatura do agente oposto
        peerDSA_PK.verify(peerSignature, peerDH_PK_Bytes, hashes.SHA256())
        print(agente + ": Assinatura validada com sucesso")
    except:
        print(agente + ": Assinatura não validada")
    
    #Obtenção da chave DH partilhada
    chavePartilhada = chavePrivadaDH.exchange(peerDH_PK)
    chaveDerivada = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        info=b'handshake data',
        backend=default_backend()
    ).derive(chavePartilhada)
    
    
    #Verificação se a chave obtida por ambos os agentes foi a mesma através de um Hmac
    hashVerificacao = mac(b'PasswordVerificacao',chaveDerivada)
    conn.send(hashVerificacao)
    peerHashChave = conn.recv()
    mac(b'PasswordVerificacao', chaveDerivada, peerHashChave)
    print(agente + ": Processo DH concluído com sucesso")
    
    return chaveDerivada

##### Comunicação Privada
* Para os agentes comunicarem entre si, são criados dois processos que comunicam entre si através de um *pipe*, permitindo assim ao ***Emitter***, após ter completado o processo de troca de chaves, enviar uma *stream* de informação encriptada ao ***Receiver*** fechando ambos a conexão após o envio/receção dessa *stream* de dados.

In [7]:
from multiprocessing import Pipe, Process

class PipeCom(object):
    def __init__(self,left,right,timeout=None): #Recebe os agentes e um timeout para os processos a serem criados
        left_end, right_end = Pipe() #São criadas as "pontas" do Pipe
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,)) #Cria um processo e define qual o agente desse processo
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end) #Define quais as "pontas" a que cada agente tem acesso
        self.right = lambda : right(right_end)
    
    def executa(self): #Cria os processos e executa os agentes
        self.lproc.start()
        self.rproc.start()  
        self.lproc.join(self.timeout)
        self.rproc.join(self.timeout)

In [8]:
mensagem = b"Mensagem a encriptar e enviar e caracteres aleatorios jkc kscskdjnc hfjshc kjfhskjcnkjs jkcvsjkhvnkj"

def Receiver(conn):
    chaveR = sts('Receiver', conn)
    
    try:#Separar os diferentes componentes da stream(dicionário) de dados recebida
        data = conn.recv() #Recebe dados
        ciphertext = data['cipher']
        iv = data['iv']
        tag = data['tag']

        print(decrypt( #Decifra o ciphertext e verifica a autenticidade dos metadados
            chaveR,
            iv,
            ciphertext,
            tag
        ))
        print("Receiver: Processo concluído com sucesso")
    except:
        print("Receiver: Erro")
        
    conn.close() #Fecha a sua "ponta" do Pipe

def Emitter(conn):
    chaveE = sts('Emitter', conn)
    
    try:
        iv, ciphertext, tag = encrypt( #Cifra texto limpo e autentica o mesmo
            chaveE,
            mensagem
        )
        data = {'cipher' : ciphertext , 'iv' : iv, 'tag' : tag} #Organiza os elementos numa Stream(dicionário)
        conn.send(data) #Envia dados
        print("Emitter: Dados Enviados")
    except:
        print("Emitter: Erro")
        
    conn.close() #Fecha a sua "ponta" do Pipe

In [9]:
PipeCom(Emitter, Receiver).executa()

Receiver: Assinatura validada com sucesso
Emitter: Assinatura validada com sucesso
Emitter: Processo DH concluído com sucesso
Receiver: Processo DH concluído com sucesso
Emitter: Dados Enviados
b'Mensagem a encriptar e enviar e caracteres aleatorios jkc kscskdjnc hfjshc kjfhskjcnkjs jkcvsjkhvnkj'
Receiver: Processo concluído com sucesso
